In [1]:
import os
import time
import shutil
import json
import requests
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.compat.v1.keras as keras
from tensorflow.keras.applications import ( 
    xception,
    vgg16,
    vgg19,
    resnet,
    resnet50,
    resnet_v2,
    inception_v3,
    inception_resnet_v2,
    mobilenet,
    densenet,
    nasnet,
    mobilenet_v2
)
from tensorflow.keras.preprocessing import image
from concurrent import futures
from itertools import compress


models = {
    'xception':xception,
#     'vgg16':vgg16,
#     'vgg19':vgg19,
    'resnet50':resnet50,
#     'resnet101':resnet,
#     'resnet152':resnet,
#     'resnet50_v2':resnet_v2,
#     'resnet101_v2':resnet_v2,
#     'resnet152_v2':resnet_v2,
#     'resnext50':resnext,
#     'resnext101':resnext,
#     'inception_v3':inception_v3,
#     'inception_resnet_v2':inception_resnet_v2,
#     'mobilenet':mobilenet,
#     'densenet121':densenet,
#     'densenet169':densenet,
#     'densenet201':densenet,
#     'nasnet':nasnet,
#     'nasnet':nasnet,
#     'mobilenet_v2':mobilenet_v2
}

models_detail = {
    'xception':xception.Xception(weights='imagenet',include_top=False),
#     'vgg16':vgg16.VGG16(weights='imagenet'),
#     'vgg19':vgg19.VGG19(weights='imagenet'),
    'resnet50':resnet50.ResNet50(weights='imagenet'),
#     'resnet101':resnet.ResNet101(weights='imagenet'),
#     'resnet152':resnet.ResNet152(weights='imagenet'),
#     'resnet50_v2':resnet_v2.ResNet50V2(weights='imagenet'),
#     'resnet101_v2':resnet_v2.ResNet101V2(weights='imagenet'),
#     'resnet152_v2':resnet_v2.ResNet152V2(weights='imagenet'),
#     'resnext50':resnext.ResNeXt50(weights='imagenet'),
#     'resnext101':resnext.ResNeXt101(weights='imagenet'),
#     'inception_v3':inception_v3.InceptionV3(weights='imagenet'),
#     'inception_resnet_v2':inception_resnet_v2.InceptionResNetV2(weights='imagenet'),
#     'mobilenet':mobilenet.MobileNet(weights='imagenet'),
#     'densenet121':densenet.DenseNet121(weights='imagenet'),
#     'densenet169':densenet.DenseNet169(weights='imagenet'),
#     'densenet201':densenet.DenseNet201(weights='imagenet'),
#     'nasnet':nasnet.NASNetLarge(weights='imagenet'),
#     'nasnet':nasnet.NASNetMobile(weights='imagenet'),
#     'mobilenet_v2':mobilenet_v2.MobileNetV2(weights='imagenet')
}


print('test')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
test


In [2]:
model_type = 'resnet50'

# https://github.com/tensorflow/tensorflow/issues/29931
temp = tf.zeros([8, 224, 224, 3])
_ = models[model_type].preprocess_input(temp)

print('test2')

test2


In [ ]:
from tensorflow.core.framework import graph_pb2
from tensorflow.python.platform import gfile

def ConvertFP32ToOther(graphdef):
      """Converts an FP32 network by casting all constants (weights) to a lower
         precision floating point type (FP16) and updating the dtypes
         everywhere."""
    cast_type = "float16"
    sess = tf.Session(graph=tf.import_graph_def(graphdef))
    output_graph_def = graph_pb2.GraphDef()
    dummy_tensor = sess.run(tf.constant([0.1]))
    dummy_tensor_proto = tensor_util.make_tensor_proto(dummy_tensor, \
          dtype=cast_type, shape=dummy_tensor.shape)
    dummy_tensor32 = sess.run(tf.constant([0.1]))
    dummy_tensor_proto32 = tensor_util.make_tensor_proto(dummy_tensor, \
          dtype=tf.float32, shape=dummy_tensor.shape)
    dt_float_type_attr = attr_value_pb2.AttrValue(type=dummy_tensor_proto32.dtype)
    dt_half_type_attr = attr_value_pb2.AttrValue(type=dummy_tensor_proto.dtype)
    for node in graphdef.node:
        output_node = node_def_pb2.NodeDef()
        output_node.CopyFrom(node)
        if (node.op == "Const"):
            if (node.attr["dtype"] == dt_float_type_attr):
                a = tensor_util.MakeNdarray(node.attr["value"].tensor)
                a = tf.cast(a, cast_type)
                a = sess.run(a)
                output_node.attr["dtype"].CopyFrom(dt_half_type_attr)
                output_node.attr["value"].CopyFrom(
                    attr_value_pb2.AttrValue(
                      tensor=tensor_util.make_tensor_proto(a,\
                        dtype=cast_type, shape=a.shape)))
        else:
            if ("T" in node.attr.keys()):
                if (output_node.attr["T"] == dt_float_type_attr):
                    output_node.attr["T"].CopyFrom(dt_half_type_attr)
            if ("Tparams" in node.attr.keys()):
                if (output_node.attr["Tparams"] == dt_float_type_attr):
                    output_node.attr["Tparams"].CopyFrom(dt_half_type_attr)
            if ("dtype" in node.attr.keys()):
                if (node.attr["dtype"] == dt_float_type_attr):
                    output_node.attr["dtype"].CopyFrom(dt_half_type_attr)
            if ("SrcT" in node.attr.keys()):
                if (node.attr["SrcT"] == dt_float_type_attr):
                    output_node.attr["SrcT"].CopyFrom(dt_half_type_attr)
            if ("DstT" in node.attr.keys()):
                if (node.attr["DstT"] == dt_float_type_attr):
                    output_node.attr["DstT"].CopyFrom(dt_half_type_attr)
            output_graph_def.node.extend([output_node])
    return output_graph_def

def load_graph(model_file):
    graph_def = tf.GraphDef()

    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())

    return graph_def

graph_f32 = load_graph('resnet50_fp32_keras_opt.pb')
graph_f16 = ConvertFP32ToOther(graph_f32)
output_xformed_graph_name = 'resnet50_fp16_keras_opt.pb'
with gfile.GFile(output_xformed_graph_name, "wb") as f:
    f.write(graph_f16.SerializeToString())